# Relaxation Mode

In [1]:
from saeno.FiniteBodyForces import FiniteBodyForces
    
# initialize the object
M = FiniteBodyForces()

In [2]:
from saeno.buildEpsilon import buildEpsilon

# provide a material model
epsilon = buildEpsilon(1645, 0.0008, 0.0075, 0.033)
M.setMaterialModel(epsilon)

EPSILON PARAMETERS 1645 0.0008 0.0075 0.033


In [3]:
import numpy as np

# define the coordinates of the nodes of the mesh
# the array has to have the shape N_v x 3
R = np.array([[0., 0., 0.],  # 0
              [0., 1., 0.],  # 1
              [1., 1., 0.],  # 2
              [1., 0., 0.],  # 3
              [0., 0., 1.],  # 4
              [1., 0., 1.],  # 5
              [1., 1., 1.],  # 6
              [0., 1., 1.]]) # 7

# define the tetrahedra of the mesh
# the array has to have the shape N_t x 4
# every entry is an index referencing a verces in R (indices start with 0)
T = np.array([[0, 1, 7, 2],
              [0, 2, 5, 3],
              [0, 4, 5, 7],
              [2, 5, 6, 7],
              [0, 7, 5, 2]])

# define if the nodes are "variable", e.g. allowed to be moved by the solver
# a boolean matrix with shape N_v
#                   0      1      2      3      4      5      6      7
var = np.array([ True,  True, False, False,  True, False, False,  True])

# the initial displacements of the nodes
# if the node is fixed (e.g. not variable) than this displacement will be fixed
# during the solving
U = np.array([[ 0.  ,  0.  ,  0.  ],  # 0
              [ 0.  ,  0.  ,  0.  ],  # 1
              [-0.25,  0.  ,  0.  ],  # 2
              [-0.25,  0.  ,  0.  ],  # 3
              [ 0.  ,  0.  ,  0.  ],  # 4
              [-0.25,  0.  ,  0.  ],  # 5
              [-0.25,  0.  ,  0.  ],  # 6
              [ 0.  ,  0.  ,  0.  ]]) # 7

In [4]:
# provide the node data
M.setMeshCoords(R, var, U)
# and the tetrahedron data
M.setMeshTets(T)

In [5]:
# relax the mesh and move the "varible" nodes
M.relax()

updateGloFAndK time 3.521620273590088 s
updateGloFAndK time 0.08078360557556152 s
Newton  0 : du= 5.215622744593346e+28   Energy= 588743156486.1687   Residuum= 8.189070100263168e+21
updateGloFAndK time 0.0738065242767334 s
Newton  1 : du= 6.33449486346705e+27   Energy= 589371042454.6727   Residuum= 8.189115885596382e+21
updateGloFAndK time 0.0827779769897461 s
Newton  2 : du= 2.483091398803401e+28   Energy= 589371042454.6727   Residuum= 8.189116371656993e+21
updateGloFAndK time 0.08078336715698242 s
Newton  3 : du= 1.2052242108362003e+28   Energy= 589371042454.6727   Residuum= 8.189117858894437e+21
updateGloFAndK time 0.0937497615814209 s
Newton  4 : du= 7.782985605939931e+27   Energy= 589371042454.6727   Residuum= 8.189118517649427e+21
updateGloFAndK time 0.15558266639709473 s
Newton  5 : du= 5.454919341254447e+27   Energy= 589371042454.6727   Residuum= 8.189118566354696e+21
updateGloFAndK time 0.10073041915893555 s
Newton  6 : du= 5.956166673697162e+27   Energy= 589371042454.6727   R

In [6]:
# store the forces of the nodes
M.storeF("F.dat")
# store the positions and the displacements
M.storeRAndU("R.dat", "U.dat")
# store the center of each tetrahedron and a combined list with energies and volumina of the tetrahedrons
M.storeEandV("RR.dat", "EV.dat")

F.dat stored.
R.dat stored.
U.dat stored.
RR.dat stored.
EV.dat stored.
